In [15]:
import logging
import openstack, chi, chi.ssh, chi.network, chi.server, chi.context, os

import psycopg2
from psycopg2.extras import RealDictCursor
import pandas as pd
from datetime import datetime
import json
from psycopg2.extras import Json

from keystoneauth1.identity import v3
from keystoneauth1 import session
from keystoneclient.v3 import client

In [16]:
import chi.lease

NameError: name 'os_conn' is not defined

In [4]:
import chi.lease

In [14]:
os_conn = openstack.connection.from_config()

AssertionError: 

In [10]:
blazar_conn = chi.clients.blazar()

In [12]:
blazar_conn.leases.get()

AttributeError: 'Client' object has no attribute 'leases'

In [2]:
# Enabling basic logging 
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('resource_tracker.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

In [8]:
chi.use_site("KVM@TACC")
PROJECT_NAME = "CHI-231138"
chi.set("project_name", PROJECT_NAME)

Now using KVM@TACC:
URL: https://kvm.tacc.chameleoncloud.org
Location: Austin, Texas, USA
Support contact: help@chameleoncloud.org


Unauthorized: Error authenticating with application credential: Application credentials cannot request a scope. (HTTP 401) (Request-ID: req-9b785eb0-7186-410b-af9e-05f2793df38c)

In [46]:
class ResourceTracker:

    def __init__(self, db_params: Dict[str,str]):
        self.db_params = db_params
        self.os_conn = openstack.connection.from_config()
    
    def get_db_connection(self):
        """Create and return a database connection"""
        return psycopg2.connect(**DB_CONFIG)

    def fetch_current_resources(self) -> Dict[str, List[Any]]:
        """Fetch all current resources from OpenStack"""
        try:
            resources = {
                'servers': list(self.os_conn.list_servers()),
                'networks': list(self.os_conn.list_networks()),
                'routers': list(self.os_conn.list_routers()),
                'subnets': list(self.os_conn.list_subnets())
            }
            return resources
        except Exception as e:
            logger.error(f"Error fetching resources: {str(e)}")
            raise

    def update_servers(self, conn, servers: List[Any], current_time: datetime):
        """Update server records in the database"""
        try:
            with conn.cursor() as cur:
                # Get existing server IDs
                cur.execute("SELECT resource_id FROM servers")
                existing_ids = {row[0] for row in cur.fetchall()}
                
                # Process each server
                current_ids = set()
                for server in servers:
                    current_ids.add(server.id)
                    
                    # Prepare server data
                    server_data = {
                        'resource_id': server.id,
                        'resource_name': server.name,
                        'status': server.status,
                        'created_time': server.created_at,
                        'updated_time': server.updated_at,
                        'last_seen_time': current_time,
                        'flavor': server.flavor.get('id') if server.flavor else None,
                        'image': server.image.get('id') if server.image else None,
                        'security_groups': [sg.get('name') for sg in server.security_groups],
                        'addresses': Json(server.addresses)
                    }
                    
                    if server.id in existing_ids:
                        # Update existing server
                        cur.execute("""
                            UPDATE servers 
                            SET resource_name = %(resource_name)s,
                                status = %(status)s,
                                updated_time = %(updated_time)s,
                                last_seen_time = %(last_seen_time)s,
                                flavor = %(flavor)s,
                                image = %(image)s,
                                security_groups = %(security_groups)s,
                                addresses = %(addresses)s
                            WHERE resource_id = %(resource_id)s
                        """, server_data)
                    else:
                        # Insert new server
                        cur.execute("""
                            INSERT INTO servers (
                                resource_id, resource_name, status, created_time, 
                                updated_time, last_seen_time, flavor, image,
                                security_groups, addresses
                            ) VALUES (
                                %(resource_id)s, %(resource_name)s, %(status)s, %(created_time)s,
                                %(updated_time)s, %(last_seen_time)s, %(flavor)s, %(image)s,
                                %(security_groups)s, %(addresses)s
                            )
                        """, server_data)
                
                # Update first_time_not_seen for servers that no longer exist
                missing_ids = existing_ids - current_ids
                if missing_ids:
                    cur.execute("""
                        UPDATE servers 
                        SET first_time_not_seen = %s
                        WHERE resource_id = ANY(%s)
                        AND first_time_not_seen IS NULL
                    """, (current_time, list(missing_ids)))
                
        except Exception as e:
            logger.error(f"Error updating servers: {str(e)}")
            raise

    def update_networks(self, conn, networks: List[Any], current_time: datetime):
        """Update network records in the database"""
        try:
            with conn.cursor() as cur:
                # Get existing network IDs
                cur.execute("SELECT resource_id FROM networks")
                existing_ids = {row[0] for row in cur.fetchall()}
                
                # Process each network
                current_ids = set()
                for network in networks:
                    current_ids.add(network.id)
                    
                    # Prepare network data
                    network_data = {
                        'resource_id': network.id,
                        'resource_name': network.name,
                        'status': network.status,
                        'created_time': network.created_at,
                        'updated_time': network.updated_at,
                        'last_seen_time': current_time,
                        'port_security_enabled': network.is_port_security_enabled
                    }
                    
                    if network.id in existing_ids:
                        # Update existing network
                        cur.execute("""
                            UPDATE networks 
                            SET resource_name = %(resource_name)s,
                                status = %(status)s,
                                updated_time = %(updated_time)s,
                                last_seen_time = %(last_seen_time)s,
                                port_security_enabled = %(port_security_enabled)s
                            WHERE resource_id = %(resource_id)s
                        """, network_data)
                    else:
                        # Insert new network
                        cur.execute("""
                            INSERT INTO networks (
                                resource_id, resource_name, status, created_time,
                                updated_time, last_seen_time, port_security_enabled
                            ) VALUES (
                                %(resource_id)s, %(resource_name)s, %(status)s, %(created_time)s,
                                %(updated_time)s, %(last_seen_time)s, %(port_security_enabled)s
                            )
                        """, network_data)
                
                # Update first_time_not_seen for networks that no longer exist
                missing_ids = existing_ids - current_ids
                if missing_ids:
                    cur.execute("""
                        UPDATE networks 
                        SET first_time_not_seen = %s
                        WHERE resource_id = ANY(%s)
                        AND first_time_not_seen IS NULL
                    """, (current_time, list(missing_ids)))
                
        except Exception as e:
            logger.error(f"Error updating networks: {str(e)}")
            raise
    def update_routers(self, conn, routers: List[Any], current_time: datetime):
        """Update router records in the database"""
        try:
            with conn.cursor() as cur:
                # Get existing router IDs
                cur.execute("SELECT resource_id FROM routers")
                existing_ids = {row[0] for row in cur.fetchall()}
                
                # Process each router
                current_ids = set()
                for router in routers:
                    current_ids.add(router.id)
                    
                    # Prepare router data
                    router_data = {
                        'resource_id': router.id,
                        'resource_name': router.name,
                        'status': router.status,
                        'created_time': router.created_at,
                        'updated_time': router.updated_at,
                        'last_seen_time': current_time,
                        'external_gateway_info': Json(router.external_gateway_info)
                    }
                    
                    if router.id in existing_ids:
                        # Update existing router
                        cur.execute("""
                            UPDATE routers 
                            SET resource_name = %(resource_name)s,
                                status = %(status)s,
                                updated_time = %(updated_time)s,
                                last_seen_time = %(last_seen_time)s,
                                external_gateway_info = %(external_gateway_info)s
                            WHERE resource_id = %(resource_id)s
                        """, router_data)
                    else:
                        # Insert new router
                        cur.execute("""
                            INSERT INTO routers (
                                resource_id, resource_name, status, created_time,
                                updated_time, last_seen_time, external_gateway_info
                            ) VALUES (
                                %(resource_id)s, %(resource_name)s, %(status)s, %(created_time)s,
                                %(updated_time)s, %(last_seen_time)s, %(external_gateway_info)s
                            )
                        """, router_data)
                
                # Update first_time_not_seen for routers that no longer exist
                missing_ids = existing_ids - current_ids
                if missing_ids:
                    cur.execute("""
                        UPDATE routers 
                        SET first_time_not_seen = %s
                        WHERE resource_id = ANY(%s)
                        AND first_time_not_seen IS NULL
                    """, (current_time, list(missing_ids)))
                
        except Exception as e:
            logger.error(f"Error updating routers: {str(e)}")
            raise

    def update_subnets(self, conn, subnets: List[Any], current_time: datetime):
        """Update subnet records in the database"""
        try:
            with conn.cursor() as cur:
                # Get existing subnet IDs
                cur.execute("SELECT resource_id FROM subnets")
                existing_ids = {row[0] for row in cur.fetchall()}
                
                # Process each subnet
                current_ids = set()
                for subnet in subnets:
                    current_ids.add(subnet.id)
                    
                    # Prepare subnet data
                    subnet_data = {
                        'resource_id': subnet.id,
                        'resource_name': subnet.name,
                        'status': 'ACTIVE',  # Subnets don't typically have a status field
                        'created_time': subnet.created_at,
                        'updated_time': subnet.updated_at,
                        'last_seen_time': current_time,
                        'network_id': subnet.network_id,
                        'allocation_pools': Json(subnet.allocation_pools),
                        'cidr': subnet.cidr
                    }
                    
                    if subnet.id in existing_ids:
                        # Update existing subnet
                        cur.execute("""
                            UPDATE subnets 
                            SET resource_name = %(resource_name)s,
                                status = %(status)s,
                                updated_time = %(updated_time)s,
                                last_seen_time = %(last_seen_time)s,
                                network_id = %(network_id)s,
                                allocation_pools = %(allocation_pools)s,
                                cidr = %(cidr)s
                            WHERE resource_id = %(resource_id)s
                        """, subnet_data)
                    else:
                        # Insert new subnet
                        cur.execute("""
                            INSERT INTO subnets (
                                resource_id, resource_name, status, created_time,
                                updated_time, last_seen_time, network_id,
                                allocation_pools, cidr
                            ) VALUES (
                                %(resource_id)s, %(resource_name)s, %(status)s, %(created_time)s,
                                %(updated_time)s, %(last_seen_time)s, %(network_id)s,
                                %(allocation_pools)s, %(cidr)s
                            )
                        """, subnet_data)
                
                # Update first_time_not_seen for subnets that no longer exist
                missing_ids = existing_ids - current_ids
                if missing_ids:
                    cur.execute("""
                        UPDATE subnets 
                        SET first_time_not_seen = %s
                        WHERE resource_id = ANY(%s)
                        AND first_time_not_seen IS NULL
                    """, (current_time, list(missing_ids)))
                
        except Exception as e:
            logger.error(f"Error updating subnets: {str(e)}")
            raise

    def update_resources(self):
        """Main method to update all resources"""
        current_time = datetime.now()
        
        try:
            # Fetch all current resources
            resources = self.fetch_current_resources()
            
            # Get database connection
            conn = self.get_db_connection()
            try:
                # Start transaction
                conn.autocommit = False
                
                # Update each resource type
                self.update_servers(conn, resources['servers'], current_time)
                self.update_networks(conn, resources['networks'], current_time)
                self.update_routers(conn, resources['routers'], current_time)
                self.update_subnets(conn, resources['subnets'], current_time)
                
                # Commit transaction
                conn.commit()
                logger.info("Successfully updated all resources")
                
            except Exception as e:
                conn.rollback()
                logger.error(f"Error in transaction, rolling back: {str(e)}")
                raise
            finally:
                conn.close()
                
        except Exception as e:
            logger.error(f"Failed to update resources: {str(e)}")
            raise

def main():
    # Database connection parameters
    db_params = {
        'dbname': 'resource_tracker', 
        'user': 'admin',           
        'password': 'Shakti!1997',  
        'host': 'localhost',
        'port': '5432'
    }
    
    tracker = ResourceTracker(db_params)
    tracker.update_resources()

if __name__ == "__main__":
    main()

2025-01-25 00:39:08,940 - __main__ - INFO - Successfully updated all resources
